In [1]:
import pandas as pd
from pathlib import Path

PATH = Path.cwd().parent.joinpath('data')

# Goal

The goal of this notebook is to explore if we can use commodity trader's data for our analysis of the shadowfleet. Do they use vessels of the shadowfleet and how have they responded to the war in Ukraine? 

In [ ]:
dfs = []
for file in PATH.joinpath('bloomberg', 'charters').glob('*.xlsx'):

    file_extension = file.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(file, engine='openpyxl', skiprows=1)
    elif file_extension == 'xls':
        df = pd.read_excel(file, skiprows=1)
    
    dfs.append(df)

fixtures = pd.concat(dfs)
fixtures.columns = fixtures.columns.str.lower()
fixtures.charterer = 'Vitol'

fixtures.dropna(subset=['imo_number'], inplace=True)
fixtures.drop_duplicates(inplace=True)
fixtures.reset_index(drop=True, inplace=True)
fixtures[['fixture_date', 'loading_date']] = fixtures[['fixture_date', 'loading_date']].apply(pd.to_datetime, errors='coerce')
fixtures.to_csv(PATH.joinpath('bloomberg', 'fixtures.csv'), index=False)

len(fixtures)

In [ ]:
uninsured = pd.read_csv(PATH.joinpath('processed', 'uninsured.csv'))
uninsured.start_date = pd.to_datetime(uninsured.start_date, errors='coerce')
uninsured.end_date = pd.to_datetime(uninsured.end_date, errors='coerce')
uninsured.earliest_sanction_date = pd.to_datetime(uninsured.earliest_sanction_date, errors='coerce')
uninsured.info()

In [ ]:
temp = pd.merge(fixtures, 
                uninsured, 
                left_on=['imo_number'],
                right_on='imo',
                how='left')
temp = temp[(temp.imo.notna()) & (temp.fixture_date >= temp.start_date) & (temp.fixture_date <= temp.end_date)].copy().reset_index(drop=True)
len(temp)

In [ ]:
# How many vessels are in both datasets?

intersection = set(fixtures[fixtures.fixture_date > '2023-01-01'].imo_number).intersection(kse.imo)
len(intersection)

In [ ]:
fixtures[fixtures.imo_number.isin(list(intersection))].imo_number.nunique()

## Berth visits

In [ ]:
dfs = []
for file in PATH.joinpath('bloomberg', 'berth_visits').glob('*.xlsx'):

    file_extension = file.suffix.lower()[1:]

    if file_extension == 'xlsx':
        df = pd.read_excel(file, engine='openpyxl', skiprows=1)
    elif file_extension == 'xls':
        df = pd.read_excel(file, skiprows=1)
    
    dfs.append(df)

visits = pd.concat(dfs)


In [ ]:
visits.drop_duplicates(inplace=True)
visits.columns = visits.columns.str.lower()

In [ ]:
visits.to_csv(PATH.joinpath('bloomberg', 'berth_visits.csv'), index=False)

In [ ]:
len(set(visits.vessel_imo).intersection(uninsured.imo))

In [ ]:
visits.arrival_time = pd.to_datetime(visits.arrival_time, errors='coerce')

In [ ]:
temp = pd.merge(visits,
                uninsured,
                left_on=['vessel_imo'],
                right_on='imo',
                how='left')

temp = temp[(temp.imo.notna()) & (temp.arrival_time >= temp.start_date) & (temp.arrival_time <= temp.end_date)].copy().reset_index(drop=True)

In [ ]:
len(temp)

In [ ]:
temp.drop_duplicates(inplace=True)
len(temp)

In [ ]:
temp[temp.earliest_sanction_date.notna() & (temp.arrival_time >= temp.earliest_sanction_date)]

## NL vessels

In [ ]:
vessels = [9183271, 9277761, 9277773, 9319870, 9334296, 9387255]

In [ ]:
visits[visits.vessel_imo.isin(vessels)]